# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     |  Valeria Guzmán Galván |
| **Fecha**      |  09/02/2026 |
| **Expediente** |  756902 |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente.

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

¿Qué puedes observar al ajustar los valores de `alpha`?

Compara los resultados de los coeficientes utilizando valores diferentes de $\alpha$ y los $R^2$ resultantes.



In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
from google.colab import files
files.upload()

Saving Advertising.csv to Advertising.csv


{'Advertising.csv': b',TV,radio,newspaper,sales\n1,230.1,37.8,69.2,22.1\n2,44.5,39.3,45.1,10.4\n3,17.2,45.9,69.3,9.3\n4,151.5,41.3,58.5,18.5\n5,180.8,10.8,58.4,12.9\n6,8.7,48.9,75,7.2\n7,57.5,32.8,23.5,11.8\n8,120.2,19.6,11.6,13.2\n9,8.6,2.1,1,4.8\n10,199.8,2.6,21.2,10.6\n11,66.1,5.8,24.2,8.6\n12,214.7,24,4,17.4\n13,23.8,35.1,65.9,9.2\n14,97.5,7.6,7.2,9.7\n15,204.1,32.9,46,19\n16,195.4,47.7,52.9,22.4\n17,67.8,36.6,114,12.5\n18,281.4,39.6,55.8,24.4\n19,69.2,20.5,18.3,11.3\n20,147.3,23.9,19.1,14.6\n21,218.4,27.7,53.4,18\n22,237.4,5.1,23.5,12.5\n23,13.2,15.9,49.6,5.6\n24,228.3,16.9,26.2,15.5\n25,62.3,12.6,18.3,9.7\n26,262.9,3.5,19.5,12\n27,142.9,29.3,12.6,15\n28,240.1,16.7,22.9,15.9\n29,248.8,27.1,22.9,18.9\n30,70.6,16,40.8,10.5\n31,292.9,28.3,43.2,21.4\n32,112.9,17.4,38.6,11.9\n33,97.2,1.5,30,9.6\n34,265.6,20,0.3,17.4\n35,95.7,1.4,7.4,9.5\n36,290.7,4.1,8.5,12.8\n37,266.9,43.8,5,25.4\n38,74.7,49.4,45.7,14.7\n39,43.1,26.7,35.1,10.1\n40,228,37.7,32,21.5\n41,202.5,22.3,31.6,16.6\n42,177,33.4

In [3]:
df = pd.read_csv("Advertising.csv")
df.head(20)

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
5,6,8.7,48.9,75.0,7.2
6,7,57.5,32.8,23.5,11.8
7,8,120.2,19.6,11.6,13.2
8,9,8.6,2.1,1.0,4.8
9,10,199.8,2.6,21.2,10.6


## Regresión sin penalización

In [6]:
X = df[["TV", "radio", "newspaper"]]
y = df["sales"]

In [7]:
lr = LinearRegression()
lr.fit(X,y)

LinearRegression()

In [8]:
print('Coeficientes:', lr.coef_)
print('Intercepto:', lr.intercept_)

r2 = lr.score(X,y)
print("R²:", r2)
print(f"El modelo explica aproximadamente el {r2*100:.1f}% del comportamiento de los datos.")

Coeficientess: [ 0.04576465  0.18853002 -0.00103749]
Intercepto: 2.938889369459412
R²: 0.8972106381789522
El modelo explica aproximadamente el 89.7% del comportamiento de los datos.


## Regresión con penalización L2

In [13]:
from sklearn.linear_model import Ridge

In [14]:
ridge = Ridge(alpha=1)
ridge.fit(X, y)

Ridge(alpha=1)

In [15]:
print("Coeficientes:", ridge.coef_)
print("Intercepto:", ridge.intercept_)

r2_ridge = ridge.score(X, y)
print("R2:", r2_ridge)
print(f"El modelo Ridge explica aproximadamente el {r2_ridge*100:.1f}% del comportamiento de los datos.")

Coeficientes: [ 0.04576464  0.1885251  -0.00103629]
Intercepto: 2.9389674583301506
R2: 0.8972106380074802
El modelo Ridge explica aproximadamente el 89.7% del comportamiento de los datos.


In [21]:
ridge = Ridge(alpha=9)
ridge.fit(X, y)

Ridge(alpha=9)

In [22]:
print("Coeficientes:", ridge.coef_)
print("Intercepto:", ridge.intercept_)

r2_ridge = ridge.score(X, y)
print("R2:", r2_ridge)
print(f"El modelo Ridge explica aproximadamente el {r2_ridge*100:.1f}% del comportamiento de los datos.")

Coeficientes: [ 0.04576463  0.18848575 -0.0010267 ]
Intercepto: 2.9395920481785627
R2: 0.8972106242958644
El modelo Ridge explica aproximadamente el 89.7% del comportamiento de los datos.


## Regresión con penalización L1

In [16]:
from sklearn.linear_model import Lasso

In [17]:
lasso = Lasso(alpha=0.1)
lasso.fit(X, y)

Lasso(alpha=0.1)

In [18]:
print("Coeficientes:", lasso.coef_)
print("Intercepto:", lasso.intercept_)

r2_lasso = lasso.score(X, y)
print("R2:", r2_lasso)
print(f"El modelo Lasso explica aproximadamente el {r2_lasso*100:.1f}% del comportamiento de los datos.")

Coeficientes: [ 0.04575172  0.18788735 -0.00066758]
Intercepto: 2.9444386596073855
R2: 0.8972068586756202
El modelo Lasso explica aproximadamente el 89.7% del comportamiento de los datos.


In [23]:
lasso = Lasso(alpha=0.9)
lasso.fit(X, y)

Lasso(alpha=0.9)

In [24]:
print("Coeficientes:", lasso.coef_)
print("Intercepto:", lasso.intercept_)

r2_lasso = lasso.score(X, y)
print("R2:", r2_lasso)
print(f"El modelo Lasso explica aproximadamente el {r2_lasso*100:.1f}% del comportamiento de los datos.")

Coeficientes: [0.04567076 0.18391738 0.        ]
Intercepto: 3.0283037917972173
R2: 0.8970560036782902
El modelo Lasso explica aproximadamente el 89.7% del comportamiento de los datos.


**¿Qué puedes observar al ajustar los valores de `alpha`?**

**Compara los resultados de los coeficientes utilizando valores diferentes de $\alpha$ y los $R^2$ resultantes.**

Al modificar el valor de alpha se observa que este controla la fuerza de la penalización sobre los coeficientes. En la regresión sin penalización, los coeficientes toman sus valores originales y el modelo explica aproximadamente el 89.7 % de la variabilidad de los datos. En la regresión Ridge, al aumentar alpha, los coeficientes se reducen ligeramente, pero ninguno se vuelve cero, y el valor de R2 se mantiene prácticamente igual. En la regresión Lasso, al incrementar alpha, algunos coeficientes se reducen hasta volverse exactamente cero.

En todos los casos, el desempeño del modelo medido por R2 cambia muy poco.